In [ ]:
import ollama
import cv2
import base64
import io
from PIL import Image
import json

def encode_frame_to_base64(frame):
    # Convert the frame to RGB (OpenCV uses BGR by default)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Convert to PIL Image
    pil_image = Image.fromarray(rgb_frame)
    
    # Save to bytes buffer
    buffer = io.BytesIO()
    pil_image.save(buffer, format="JPEG")
    
    # Encode to base64
    return base64.b64encode(buffer.getvalue()).decode('utf-8')

def process_video(video_path, frame_interval=60):
    client = ollama.Client()
    cap = cv2.VideoCapture(video_path)
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        if frame_count % frame_interval == 0:
            base64_frame = encode_frame_to_base64(frame)
            
            response = client.generate(
                model="llava",
                prompt="Describe this frame from a video which is a reel. Also please suggest ways to make this reel more interesting",
                images=[base64_frame]
            )
            
            
            print(f"Frame {frame_count} description: {response} type_response: {type(response)}")

    cap.release()

# Usage
video_path = "/Users/saurabhkaul/Downloads/Video-635.mov"
process_video(video_path)